In [74]:

import base64
import io
import math
import operator
import random
import time
import urllib

In [75]:
import matplotlib.pyplot as plt


In [76]:
from PIL import Image


In [77]:
import rtree

In [78]:
import shapefile


In [79]:
from shapely.geometry import Polygon, Point


In [80]:
import pyproj


In [81]:
def slice_at(plist, indices):
    '''
    Slices the list ll at the indices indicated.
    '''
    indices.append(len(plist))
    for fro, to in zip(indices, indices[1:]):
        yield plist[fro:to]

In [82]:

def plot_country(name, parts, center, radius):
    fig, ax = plt.subplots(figsize=(1.8, 1.8))

    circle = plt.Circle(center, radius, color='k', fill=False)
    for part in parts:
        #color = random.choice('bgrcmk')
        color = 'k'
        for point1, point2 in zip(part, part[1:]):
            ax.plot(
                [point1[0], point2[0]],
                [point1[1], point2[1]],
                color+'-')
    ax.add_artist(circle)
    ax.set_aspect('equal', 'datalim')
    imgdata = io.BytesIO()
    #ax.set_title(name)
    plt.axis('off')
    #plt.savefig(name.lower().replace(' ', '-') + '.png', transparent=True)
    #plt.show()
    plt.savefig('tmp.png', format='png', transparent=True)
    img = Image.open('tmp.png')
    png_info = img.info
    img.convert('P').save(imgdata, 'PNG', **png_info)
    imgdata.seek(0)
    plt.close(fig)
    return imgdata

In [83]:
def neighbours(center, radius, delta):
    cmov = delta
    rmov = delta
    cms = [cmov*20, cmov, 0, -cmov, -cmov*20]
    rms = [rmov*20, rmov, 0, -rmov, -rmov*20]
    for cx_mov in cms:
        for cy_mov in cms:
            for r_mov in rms:
                if radius + r_mov > 0:
                    yield ((center[0] + cx_mov, center[1] + cy_mov),
                           radius + r_mov)

In [84]:
def maxmin(arr):
    _max = arr[0]
    _min = arr[0]
    for x in arr:
        if x > _max:
            _max = x
        elif x < _min:
            _min = x
    return _max, _min

In [85]:

def heuristic(polygons, index, c, r, delta, iters=200):
    best_coeff = 0
    parts_area = sum(map(lambda x: x.area, polygons))
    is_ = 0
    for _ in range(iters):
        is_ += 1
        mejoro = False
        for c_, r_ in neighbours(c, r, delta):
            circle = Point(c_).buffer(r_)
            indices = range(len(polygons))
            if circle.bounds != ():
                indices = [int(i) for i in index.intersection(circle.bounds)]
            else:
                print('WARNING: circle.bounds == (). '\
                      'center = %s, radius = %.3f' % (str(c_), r_))
            intersection_area = sum(
                [polygons[i].intersection(circle).area for i in indices])
            coeff = intersection_area / max(math.pi * r_ * r_, parts_area)
            #print(c_, r_)
            #print(intersection_area, circle_area, parts_area)
            if coeff > best_coeff:
                mejoro = True
                c = c_
                r = r_
                best_coeff = coeff
        if not mejoro:
            break
    return c, r, best_coeff, is_

In [86]:
def circle_inside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = min((xmax - xmin) / 2, (ymax - ymin) / 2)
    return center, radius

In [87]:
def circle_outside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = math.sqrt((xmax - xmin)**2.0 + (ymax - ymin)**2.0) / 2
    return center, radius

In [88]:

def analyze_both_circles(polys, index, min_x, min_y, max_x, max_y):
    cinside, rinside = circle_inside(min_x, min_y, max_x, max_y)
    coutside, routside = circle_outside(min_x, min_y, max_x, max_y)
    c_i, r_i, best_i, iin = heuristic(
        polys, index, cinside, rinside, rinside / 200)
    print('[%d, ' % iin, end='', flush=True)
    c_o, r_o, best_o, iout = heuristic(
        polys, index, coutside, routside, routside / 200)
    print('%d]' % iout)
    best_coeff_ = max(best_i, best_o)
    center_ = c_i if best_i > best_o else c_o
    radius_ = r_i if best_i > best_o else r_o
    return center_, radius_, best_coeff_

In [89]:

def find_best_circle(parts, minx, miny, maxx, maxy):
    polys = [Polygon(part).buffer(0) for part in parts]
    index = rtree.index.Index()
    for i, poly in enumerate(polys):
        index.insert(i, poly.bounds)
    print('Analyzing best overall circle...  Iterations: ', end='', flush=True)
    c, r, best_coeff = analyze_both_circles(
        polys, index, minx, miny, maxx, maxy)
    best_polygons = []
    if len(polys) > 1:
        polygons_area = sorted(
            [(poly, poly.area) for poly in polys],
            key=lambda x: x[1], reverse=True)
        areas = list(map(lambda r: r[1], polygons_area))
        if areas[0] > sum(areas[1:]):
            best_polygons = [polygons_area[0][0]]
        else:
            best_polygons = list(map(lambda x: x[0], polygons_area))[:5]
    i = 0
    for part in best_polygons:
        if len(polys) == 1:
            break
        i += 1
        print('Analyzing best part circle... (%d out of %d) Iterations: ' \
            % (i, len(polys)), end='', flush=True)
        x, y = part.exterior.coords.xy
        max_x, min_x = maxmin(x)
        max_y, min_y = maxmin(y)
        center_, radius_, best_coeff_ = analyze_both_circles(
            polys, index, min_x, min_y, max_x, max_y)
        if best_coeff_ > best_coeff:
            c = center_
            r = radius_
            best_coeff = best_coeff_
    # fine tune it.
    print('Fine-tuning the circle... Iterations: ', end='', flush=True)
    c, r, best_coeff, iters = heuristic(polys, index, c, r, r / 1000, 1000)
    print('[%d]' % iters)
    return c, r, best_coeff


In [90]:
def tuple2list(t):
    t = tuple(t)[0]
    return [t[0], t[1]]

In [91]:
def concat(ls):
    return [j for i in ls for j in i]

In [92]:

def mean(ls):
    return float(sum(ls)) / len(ls)

In [93]:
INPUT_FILE = 'ne_10m_admin_0_sovereignty'
FROM_PROJ = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'


In [94]:
fields =shapefile.Reader(INPUT_FILE).fields[1:] 
field_names = [field[0] for field in fields] 
# construction of a dctionary field_name:value  
for r in shapefile.Reader(INPUT_FILE).shapeRecords():  
     atr = dict(zip(field_names, r.record))

In [95]:
print(field_names)

['featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT', 'SOV_A3', 'ADM0_DIF', 'LEVEL', 'TYPE', 'ADMIN', 'ADM0_A3', 'GEOU_DIF', 'GEOUNIT', 'GU_A3', 'SU_DIF', 'SUBUNIT', 'SU_A3', 'BRK_DIFF', 'NAME', 'NAME_LONG', 'BRK_A3', 'BRK_NAME', 'BRK_GROUP', 'ABBREV', 'POSTAL', 'FORMAL_EN', 'FORMAL_FR', 'NAME_CIAWF', 'NOTE_ADM0', 'NOTE_BRK', 'NAME_SORT', 'NAME_ALT', 'MAPCOLOR7', 'MAPCOLOR8', 'MAPCOLOR9', 'MAPCOLOR13', 'POP_EST', 'POP_RANK', 'GDP_MD_EST', 'POP_YEAR', 'LASTCENSUS', 'GDP_YEAR', 'ECONOMY', 'INCOME_GRP', 'WIKIPEDIA', 'FIPS_10_', 'ISO_A2', 'ISO_A3', 'ISO_A3_EH', 'ISO_N3', 'UN_A3', 'WB_A2', 'WB_A3', 'WOE_ID', 'WOE_ID_EH', 'WOE_NOTE', 'ADM0_A3_IS', 'ADM0_A3_US', 'ADM0_A3_UN', 'ADM0_A3_WB', 'CONTINENT', 'REGION_UN', 'SUBREGION', 'REGION_WB', 'NAME_LEN', 'LONG_LEN', 'ABBREV_LEN', 'TINY', 'HOMEPART', 'MIN_ZOOM', 'MIN_LABEL', 'MAX_LABEL', 'NE_ID', 'WIKIDATAID', 'NAME_AR', 'NAME_BN', 'NAME_DE', 'NAME_EN', 'NAME_ES', 'NAME_FR', 'NAME_EL', 'NAME_HI', 'NAME_HU', 'NAME_ID', 'NAME_IT', 'NAME_JA', 'NAM

In [96]:

def main():
    countries = shapefile.Reader(INPUT_FILE).shapeRecords()
    results = {}

    for country in countries:
        t0 = time.process_time()
        name = country.record[8]
        # if name != 'Nauru': continue
        print('==== %s (%d -> %d) ====' % (
            name, len(country.shape.parts), len(country.shape.points)))
        points = list(map(lambda x: x, country.shape.points))

        parts_ = list(country.shape.parts) + [len(country.shape.points)]
        largest_part = (0, 1)
        for x1, x2 in zip(parts_, parts_[1:]):
            if x2 - x1 > largest_part[1] - largest_part[0]:
                largest_part = (x1, x2)
        print(parts_, largest_part)

        # Get the random points from the largest part.
        random_points = list(
            map(
                list,
                random.sample(points[largest_part[0]:largest_part[1]], 2)))

        # Get the middle points between them.
        lon, lat = list(
            pyproj.Geod(ellps='WGS84').npts(
                random_points[0][0], random_points[0][1],
                random_points[1][0], random_points[1][1],
                1)  # Just 1 point in-between them.
            )[0]

        to_proj = '+proj=aeqd +R=6371000 +lat_0=%.2f +lon_0=%.2f ' \
                  '+no_defs' % (lat, lon)
        print('Projection: %s' % to_proj)

        transform = lambda g: pyproj.transform(
            pyproj.Proj(FROM_PROJ),
            pyproj.Proj(to_proj),
            g[0], g[1])
        points = list(map(transform, points))

        parts = list(slice_at(points, country.shape.parts))
        max_xs, min_xs = maxmin(list(map(lambda t: t[0], points)))
        max_ys, min_ys = maxmin(list(map(lambda t: t[1], points)))

        center, radius, best_coeff = find_best_circle(
            parts, min_xs, min_ys, max_xs, max_ys)

        print('Time to compute roundness: %.3fs.' % (time.process_time() - t0))
        print('Result info: %f (%.4f, %.4f) %.4f' \
            % (best_coeff, center[0], center[1], radius))
        uri = 'data:image/png;base64,' + urllib.parse.quote(
            base64.b64encode(
                plot_country(name, parts, center, radius).getvalue()))
        print('Finished plotting %s.' % name)
        results[name] = (best_coeff, '<img src="%s" />' % uri)

    f = open('table', 'w')
    # Does python use theorems for free?
    # (https://www.mpi-sws.org/~dreyer/tor/papers/wadler.pdf)
    f.write(
        '\n'.join(
            map(lambda s: ' | '.join(map(str, s)),
                map(lambda r: [r[0]+1, r[1][0],
                               '%.3f' % r[1][1][0],
                               r[1][1][1]],
                    enumerate(
                        sorted(results.items(),
                               key=lambda z: (operator.itemgetter(1)(z))[0],
                               reverse=True))))))
    f.close()


In [97]:
main()

==== Indonesia                        (264 -> 19565) ====
[0, 17, 261, 2099, 4079, 4095, 4121, 4145, 4166, 4181, 4212, 4228, 4253, 4280, 4316, 4334, 4357, 4388, 4415, 4431, 4455, 4500, 4522, 4580, 5474, 5494, 5525, 5540, 5557, 5571, 5586, 5605, 5618, 5637, 5658, 5681, 5703, 5734, 5748, 5762, 5771, 5781, 5789, 5809, 5830, 5845, 5862, 5871, 5890, 5905, 5921, 5935, 5946, 5960, 5971, 5984, 7776, 7788, 7804, 9247, 9415, 9433, 9482, 9558, 9586, 9739, 9752, 10188, 10568, 10664, 10687, 10750, 10777, 10810, 10823, 10843, 10940, 10949, 11031, 11081, 11145, 11175, 11218, 11340, 11399, 11428, 11449, 11464, 11480, 11506, 11524, 11546, 11565, 11584, 11619, 11676, 11711, 11772, 11860, 11902, 11935, 11958, 11975, 12012, 12025, 12063, 12087, 12115, 12130, 12253, 12280, 12290, 12349, 12368, 12389, 12416, 12439, 12456, 12486, 12503, 12517, 12615, 12638, 12655, 12687, 12719, 12796, 12828, 12866, 13002, 13062, 13090, 13188, 13220, 13270, 13300, 13414, 13445, 13525, 13719, 13742, 13767, 13864, 13893, 13990,

Analyzing best part circle... (1 out of 35) Iterations: [16, 12]
Fine-tuning the circle... Iterations: [4]
Time to compute roundness: 73.229s.
Result info: 0.767130 (-182735.4191, 83452.5658) 1004550.4584
Finished plotting India                           .
==== China                            (75 -> 14635) ====
[0, 12171, 12842, 12876, 12902, 12944, 12967, 12988, 13017, 13052, 13091, 13115, 13156, 13186, 13215, 13229, 13257, 13272, 13286, 13297, 13343, 13352, 13368, 13428, 13477, 13507, 13575, 13593, 13637, 13676, 13702, 13768, 13813, 13823, 13829, 13839, 13850, 13860, 13871, 13880, 13893, 13901, 13921, 13932, 13940, 13948, 13957, 13979, 13998, 14007, 14022, 14038, 14085, 14130, 14150, 14213, 14280, 14294, 14313, 14324, 14342, 14376, 14402, 14413, 14431, 14447, 14474, 14492, 14512, 14529, 14542, 14569, 14596, 14612, 14631, 14635] (0, 12171)
Projection: +proj=aeqd +R=6371000 +lat_0=24.27 +lon_0=115.30 +no_defs
Analyzing best overall circle...  Iterations: [9, 18]
Analyzing best part ci

Analyzing best overall circle...  Iterations: [9, 16]
Analyzing best part circle... (1 out of 53) Iterations: [10, 9]
Fine-tuning the circle... Iterations: [4]
Time to compute roundness: 47.006s.
Result info: 0.818638 (95394.0332, 90443.0917) 177185.6246
Finished plotting South Korea                     .
==== North Korea                      (10 -> 1782) ====
[0, 1613, 1639, 1654, 1674, 1688, 1709, 1721, 1739, 1751, 1782] (0, 1613)
Projection: +proj=aeqd +R=6371000 +lat_0=40.03 +lon_0=128.43 +no_defs
Analyzing best overall circle...  Iterations: [9, 15]
Analyzing best part circle... (1 out of 10) Iterations: [11, 15]
Fine-tuning the circle... Iterations: [6]
Time to compute roundness: 28.308s.
Result info: 0.724528 (-135833.0926, -23540.3678) 197352.5587
Finished plotting North Korea                     .
==== Morocco                          (1 -> 1019) ====
[0, 1019] (0, 1019)
Projection: +proj=aeqd +R=6371000 +lat_0=34.03 +lon_0=-7.37 +no_defs
Analyzing best overall circle...  Iter

Fine-tuning the circle... Iterations: [2]
Time to compute roundness: 6.970s.
Result info: 0.855585 (-34085.5538, 6511.2425) 143396.2574
Finished plotting Lithuania                       .
==== Brazil                           (43 -> 11116) ====
[0, 9149, 9195, 9234, 9277, 9308, 9345, 9372, 9418, 9447, 9471, 9493, 9534, 9561, 9669, 9706, 10262, 10306, 10347, 10386, 10397, 10460, 10519, 10557, 10581, 10606, 10638, 10661, 10670, 10679, 10742, 10780, 10808, 10834, 10882, 10959, 10975, 10988, 11006, 11030, 11048, 11079, 11095, 11116] (0, 9149)
Projection: +proj=aeqd +R=6371000 +lat_0=-11.64 +lon_0=-64.12 +no_defs
Analyzing best overall circle...  Iterations: [13, 13]
Analyzing best part circle... (1 out of 43) Iterations: [11, 17]
Fine-tuning the circle... Iterations: [6]
Time to compute roundness: 134.506s.
Result info: 0.803431 (1268649.7096, 64156.4411) 1656595.7859
Finished plotting Brazil                          .
==== Uruguay                          (1 -> 581) ====
[0, 581] (0, 581)

Finished plotting Finland                         .
==== Vietnam                          (25 -> 4166) ====
[0, 3511, 3544, 3561, 3587, 3613, 3652, 3680, 3713, 3726, 3765, 3818, 3901, 3925, 3946, 3963, 3990, 4010, 4061, 4073, 4095, 4104, 4120, 4134, 4157, 4166] (0, 3511)
Projection: +proj=aeqd +R=6371000 +lat_0=16.37 +lon_0=107.74 +no_defs
Analyzing best overall circle...  Iterations: [16, 24]
Analyzing best part circle... (1 out of 25) Iterations: [16, 24]
Fine-tuning the circle... Iterations: [15]
Time to compute roundness: 70.565s.
Result info: 0.450850 (-290683.0240, 453544.5908) 324189.3058
Finished plotting Vietnam                         .
==== Cambodia                         (6 -> 1209) ====
[0, 1090, 1106, 1136, 1165, 1186, 1209] (0, 1090)
Projection: +proj=aeqd +R=6371000 +lat_0=13.67 +lon_0=103.58 +no_defs
Analyzing best overall circle...  Iterations: [12, 15]
Analyzing best part circle... (1 out of 6) Iterations: [12, 15]
Fine-tuning the circle... Iterations: [3]
Time to c

...

Analyzing best overall circle...  Iterations: [11, 11]
Analyzing best part circle... (1 out of 10) Iterations: [12, 19]
Fine-tuning the circle... Iterations: [3]
Time to compute roundness: 10.902s.
Result info: 0.671797 (-5036.4814, 109440.3845) 84386.0048
Finished plotting Belize                          .
==== Panama                           (15 -> 1999) ====
[0, 1604, 1624, 1662, 1734, 1752, 1770, 1799, 1811, 1825, 1858, 1873, 1939, 1957, 1979, 1999] (0, 1604)
Projection: +proj=aeqd +R=6371000 +lat_0=7.99 +lon_0=-80.04 +no_defs
Analyzing best overall circle...  Iterations: [18, 15]
Analyzing best part circle... (1 out of 15) Iterations: [18, 15]
Fine-tuning the circle... Iterations: [3]
Time to compute roundness: 31.774s.
Result info: 0.503305 (-121507.6716, 57546.5630) 154406.3746
Finished plotting Panama                          .
==== Venezuela                        (29 -> 3201) ====
[0, 2658, 2690, 2714, 2745, 2761, 2780, 2796, 2823, 2903, 2918, 2929, 2946, 2962, 2974, 2985, 2

Analyzing best overall circle...  Iterations: [18, 16]
Analyzing best part circle... (1 out of 101) Iterations: [9, 12]
Fine-tuning the circle... Iterations: [3]
Time to compute roundness: 168.307s.
Result info: 0.807621 (565466.8838, -1192884.4636) 1577106.1960
Finished plotting Australia                       .
==== Fiji                             (44 -> 1476) ====
[0, 9, 35, 60, 71, 92, 479, 495, 504, 518, 533, 543, 556, 587, 608, 716, 733, 757, 773, 790, 820, 837, 852, 871, 884, 1203, 1220, 1240, 1277, 1294, 1306, 1334, 1349, 1362, 1371, 1388, 1397, 1408, 1416, 1427, 1440, 1449, 1458, 1467, 1476] (92, 479)
Projection: +proj=aeqd +R=6371000 +lat_0=-16.71 +lon_0=179.01 +no_defs
Analyzing best overall circle...  Iterations: [19, 19]
Analyzing best part circle... (1 out of 44) Iterations: [15, 7]
Fine-tuning the circle... Iterations: [2]
Time to compute roundness: 20.655s.
Result info: 0.578185 (-108718.1260, -139301.2381) 77708.2999
Finished plotting Fiji                            .

Analyzing best part circle... (2 out of 35) Iterations: [4, 2]
Analyzing best part circle... (3 out of 35) Iterations: [5, 2]
Analyzing best part circle... (4 out of 35) Iterations: [13, 2]
Analyzing best part circle... (5 out of 35) Iterations: [16, 2]
Fine-tuning the circle... Iterations: [3]
Time to compute roundness: 10.566s.
Result info: 0.350889 (-15506.4783, 2277.9013) 17551.4850
Finished plotting Kiribati                        .
==== Marshall Islands                 (22 -> 211) ====
[0, 13, 23, 35, 43, 62, 72, 86, 103, 115, 123, 130, 134, 145, 160, 164, 172, 183, 188, 193, 199, 204, 211] (43, 62)
Projection: +proj=aeqd +R=6371000 +lat_0=5.85 +lon_0=169.63 +no_defs
Analyzing best overall circle...  Iterations: WARNING: circle.bounds == (). center = (230226.51490439472, 162870.03627829626), radius = 0.000
[28, 29]
Analyzing best part circle... (1 out of 22) Iterations: [14, 16]
Analyzing best part circle... (2 out of 22) Iterations: [8, 14]
Analyzing best part circle... (3 out o

Finished plotting Comoros                         .
==== São Tomé and Principe          (2 -> 96) ====
[0, 54, 96] (0, 54)
Projection: +proj=aeqd +R=6371000 +lat_0=0.24 +lon_0=6.61 +no_defs
Analyzing best overall circle...  Iterations: [22, 19]
Analyzing best part circle... (1 out of 2) Iterations: [12, 10]
Fine-tuning the circle... Iterations: [4]
Time to compute roundness: 3.872s.
Result info: 0.787632 (633.3301, 93.7104) 18213.0601
Finished plotting São Tomé and Principe         .
==== Cabo Verde                       (9 -> 409) ====
[0, 23, 49, 144, 190, 235, 275, 318, 357, 409] (49, 144)
Projection: +proj=aeqd +R=6371000 +lat_0=15.19 +lon_0=-23.65 +no_defs
Analyzing best overall circle...  Iterations: [18, 6]
Analyzing best part circle... (1 out of 9) Iterations: [6, 18]
Analyzing best part circle... (2 out of 9) Iterations: [11, 4]
Analyzing best part circle... (3 out of 9) Iterations: [3, 2]
Analyzing best part circle... (4 out of 9) Iterations: [3, 2]
Analyzing best part circle

Finished plotting Vanuatu                         .
==== Palau                            (9 -> 165) ====
[0, 16, 46, 62, 126, 135, 140, 149, 157, 165] (62, 126)
Projection: +proj=aeqd +R=6371000 +lat_0=7.59 +lon_0=134.55 +no_defs
Analyzing best overall circle...  Iterations: [21, 23]
Analyzing best part circle... (1 out of 9) Iterations: [13, 18]
Fine-tuning the circle... Iterations: [3]
Time to compute roundness: 6.849s.
Result info: 0.554287 (3541.8991, -10360.7459) 12541.3770
Finished plotting Palau                           .
==== Bahrain                          (1 -> 50) ====
[0, 50] (0, 50)
Projection: +proj=aeqd +R=6371000 +lat_0=26.09 +lon_0=50.54 +no_defs
Analyzing best overall circle...  Iterations: [9, 9]
Fine-tuning the circle... Iterations: [6]
Time to compute roundness: 2.032s.
Result info: 0.649275 (-42.6197, -3152.7662) 13652.5928
Finished plotting Bahrain                         .
==== Spratly Islands                  (12 -> 102) ====
[0, 4, 16, 23, 32, 40, 48, 56, 6